In [55]:
#note: http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/
import glob
import pandas as pd
import numpy as np

# The * is not a regex, it just means "match anything"
# This matches State_County_Penetration_MA_2017_12.csv,State_County_Penetration_MA_2017_09.csv etc.
filenames = glob.glob("./data/time_series/State_County_Penetration_*.csv")

data_types = {'FIPSST': object, 
              'FIPSCNTY': object, 
              'FIPS': object, 
              'SSAST': object, 
              'SSACNTY': object, 
              'SSA': object
             }

#creates a list of dfs, one for each file
list_of_dfs = [pd.read_csv(filename, dtype=data_types, thousands=r',', na_values=["*"]) for filename in filenames]

# zip loops through TWO THINGS AT ONCE
# so you're looking at dataframe #1 and filename #1
# then dataframe #2 and filename #2
# etc
# and assigning that filename as a new column in the dataframe
for dataframe, filename in zip(list_of_dfs, filenames):
    dataframe['filename'] = filename
    
# Combine a list of dataframes, on top of each other
ts_df = pd.concat(list_of_dfs, ignore_index=True)

#creating string date column from filename
ts_df['date'] = ts_df['filename'].apply(lambda x: x.split('Penetration_MA_')[1][0:7])

#create new date+FIPS unique ID
ts_df['d_fips'] = ts_df['date']+'-'+ts_df['FIPS']

#create new datetime column
ts_df['date'] = pd.to_datetime(ts_df['date'], format='%Y_%m')

#new calc for the pop_penetration
ts_df['pop_Penetration'] = ts_df['Enrolled']/ts_df['Eligibles']

#dropping columns we don't need
ts_df.drop(['filename', 'SSAST','SSACNTY','SSA','Penetration'] ,axis = 1, inplace=True)

#cleaning column white space
ts_df = ts_df.rename(columns={'State Name': 'state', 'County Name': 'county'})

# Resetting index so merge is easy
ts_df.set_index('d_fips', inplace = True)

#getting rid of weird non state designation
ts_df = ts_df[ts_df.state != 'Pending State Designation']
ts_df = ts_df[ts_df.county != 'Pending County Designation']

#create new year + FIPS ID for rate info
ts_df['y_fips'] = ts_df['date'].astype('str')
ts_df['y_fips'] = ts_df['y_fips'].str.slice(start=0, stop=4)+'-'+ts_df['FIPS']


ts_df.head()

,state,county,FIPSST,FIPSCNTY,FIPS,Eligibles,Enrolled,date,pop_Penetration,y_fips
d_fips,,,,,,,,,,
2014_09-01001,Alabama,Autauga,01,001,01001,9612,3049.0,2014-09-01,0.317208,2014-01001
2014_09-01003,Alabama,Baldwin,01,003,01003,42730,13573.0,2014-09-01,0.317646,2014-01003
2014_09-01005,Alabama,Barbour,01,005,01005,6097,896.0,2014-09-01,0.146958,2014-01005
2014_09-01007,Alabama,Bibb,01,007,01007,4647,1690.0,2014-09-01,0.363675,2014-01007
2014_09-01009,Alabama,Blount,01,009,01009,11385,4719.0,2014-09-01,0.414493,2014-01009


In [13]:
# The * is not a regex, it just means "match anything"
# This matches State_County_Penetration_MA_2017_12.csv,State_County_Penetration_MA_2017_09.csv etc.
filenames = glob.glob("./data/time_series/SCC_Enrollment_MA_*.csv")

data_types = {'FIPS Code': object
             }

#creates a list of dfs, one for each file
list_of_dfs = [pd.read_csv(filename, dtype=data_types, thousands=r',', na_values=["*"]) for filename in filenames]

# zip loops through TWO THINGS AT ONCE
# so you're looking at dataframe #1 and filename #1
# then dataframe #2 and filename #2
# etc
# and assigning that filename as a new column in the dataframe
for dataframe, filename in zip(list_of_dfs, filenames):
    dataframe['filename'] = filename
    
# Combine a list of dataframes, on top of each other
plan_enroll_df = pd.concat(list_of_dfs, ignore_index=True)

#creating string date column from filename
plan_enroll_df['date'] = plan_enroll_df['filename'].apply(lambda x: x.split('SCC_Enrollment_MA_')[1][0:7])

#create new date+FIPS unique ID
plan_enroll_df['d_fips'] = plan_enroll_df['date']+'-'+plan_enroll_df['FIPS Code']

#make sure col names aren't confusing after merge
plan_enroll_df.rename(columns={'Enrolled':'plan_Enrolled'}, inplace=True)

#creates df with 6,081,762 rows

In [16]:
#function for calculating the HHI index for each county/date.  https://www.investopedia.com/terms/h/hhi.asp
def hhi(df):
    temp_df = pd.DataFrame(df[df['plan_Enrolled'] > 0].groupby(['d_fips','Organization Name'])['plan_Enrolled'].sum())
    temp_df.reset_index(inplace = True)
    temp_df.set_index('d_fips', inplace = True)
    temp2 = temp_df.join(ts_df['Enrolled'], how='left')
    temp2['plan_mkt_share'] = temp2['plan_Enrolled'] / temp2['Enrolled']
    temp2['sq_mkt_share'] = (temp2['plan_mkt_share']*100)**2
    temp3 = pd.DataFrame(temp2.groupby('d_fips')['sq_mkt_share'].sum())
    temp3.rename(columns={'sq_mkt_share':'HHI_Index'}, inplace=True)
    return temp3

hhi_df = hhi(plan_enroll_df)

In [18]:
#need to get a crosswalk to change between  SSA and FIPS
data_types = {'SSA State county code': object,
              'FIPS State county code': object  
             }

crosswalk = pd.read_excel('./data/time_series/CBSAtoCountycrosswalk_FY15_FR.xlsx', dtype=data_types)
crosswalk = crosswalk[['SSA State county code', 'FIPS State county code']]
crosswalk.columns=['SSA', 'FIPS']
crosswalk.head()

,SSA,FIPS
0,01000,01001
1,01010,01003
2,01020,01005
3,01030,01007
4,01040,01009


In [47]:
# Building payment rate dfs.  Note, there is no data from 2014

#2015
df15 = pd.read_csv("./data/time_series/CountyRate2015.csv", dtype=data_types, thousands=r',', na_values=["*"], skiprows=2)
df15 = df15.merge(crosswalk, how='left', left_on='Code', right_on='SSA')
df15 = df15[['FIPS', ' 5% Bonus 2015 Rate', ' 3.5% Bonus 2015 Rate', ' 0% Bonus 2015 Rate']]
df15.columns=['FIPS', "5_pct_bonus", "3.5_pct_bonus", "0_pct_bonus"]
df15['y_fips'] = '2015'+ '-' + df15['FIPS'] 

#2016
# note, for some reason, they did not change the year in the headers for this csv
df16 = pd.read_csv("./data/time_series/CountyRate2016.csv", dtype=data_types, thousands=r',', na_values=["*"], skiprows=3)
df16 = df16.merge(crosswalk, how='left', left_on='Code', right_on='SSA')
df16 = df16[['FIPS', ' 5% Bonus 2015 Rate', '3.5% Bonus 2015 Rate', ' 0% Bonus 2015 Rate']]
df16.columns=['FIPS', "5_pct_bonus", "3.5_pct_bonus", "0_pct_bonus"]
df16['y_fips'] = '2016'+ '-' + df16['FIPS'] 

#2017
df17 = pd.read_csv("./data/time_series/CountyRate2017.csv", dtype=data_types, thousands=r',', na_values=["*"], skiprows=1)
df17 = df17.merge(crosswalk, how='left', left_on='Code', right_on='SSA')
df17 = df17[['FIPS', 'Parts A&B  5% Bonus 2017 Rate', 'Parts A&B  3% Bonus 2017 Rate', 'Parts A&B  0% Bonus 2017 Rate']]
df17.columns=['FIPS', "5_pct_bonus", "3.5_pct_bonus", "0_pct_bonus"]
df17['y_fips'] = '2017'+ '-' + df17['FIPS'] 

# Combine a list of dataframes, on top of each other
rate_df = pd.concat([df15, df16, df17], ignore_index=True)

rate_df.head()

,FIPS,5_pct_bonus,3.5_pct_bonus,0_pct_bonus,y_fips
0,01001,781.67,760.35,710.61,2015-01001
1,01003,793.71,772.78,723.94,2015-01003
2,01005,782.79,782.79,760.28,2015-01005
3,01007,801.48,787.29,754.18,2015-01007
4,01009,797.12,775.38,724.65,2015-01009


In [56]:
# join hhi_df to main ts_df
ts_df = ts_df.join(hhi_df, how='left')

ts_df.head()

,state,county,FIPSST,FIPSCNTY,FIPS,Eligibles,Enrolled,date,pop_Penetration,y_fips,HHI_Index
d_fips,,,,,,,,,,,
2014_09-01001,Alabama,Autauga,01,001,01001,9612,3049.0,2014-09-01,0.317208,2014-01001,2494.719193
2014_09-01003,Alabama,Baldwin,01,003,01003,42730,13573.0,2014-09-01,0.317646,2014-01003,2061.361924
2014_09-01005,Alabama,Barbour,01,005,01005,6097,896.0,2014-09-01,0.146958,2014-01005,5958.114935
2014_09-01007,Alabama,Bibb,01,007,01007,4647,1690.0,2014-09-01,0.363675,2014-01007,2950.456917
2014_09-01009,Alabama,Blount,01,009,01009,11385,4719.0,2014-09-01,0.414493,2014-01009,3086.813525


In [75]:
# Join rate info to main ts_df 
ts_df = ts_df.merge(rate_df, how='left')
ts_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 51406 entries, 0 to 51405
Data columns (total 14 columns):
state              51406 non-null object
county             51406 non-null object
FIPSST             51406 non-null object
FIPSCNTY           51406 non-null object
FIPS               51406 non-null object
Eligibles          51406 non-null int64
Enrolled           49793 non-null float64
date               51406 non-null datetime64[ns]
pop_Penetration    49793 non-null float64
y_fips             51406 non-null object
HHI_Index          48891 non-null float64
5_pct_bonus        38446 non-null float64
3.5_pct_bonus      38446 non-null float64
0_pct_bonus        38446 non-null float64
dtypes: datetime64[ns](1), float64(6), int64(1), object(6)
memory usage: 5.9+ MB


In [79]:
# ts_df.to_csv('MA_time_series_by_county.csv')
ts_df.head()

,state,county,FIPSST,FIPSCNTY,FIPS,Eligibles,Enrolled,date,pop_Penetration,y_fips,HHI_Index,5_pct_bonus,3.5_pct_bonus,0_pct_bonus
0,Alabama,Autauga,01,001,01001,9612,3049.0,2014-09-01,0.317208,2014-01001,2494.719193,NaN,NaN,NaN
1,Alabama,Baldwin,01,003,01003,42730,13573.0,2014-09-01,0.317646,2014-01003,2061.361924,NaN,NaN,NaN
2,Alabama,Barbour,01,005,01005,6097,896.0,2014-09-01,0.146958,2014-01005,5958.114935,NaN,NaN,NaN
3,Alabama,Bibb,01,007,01007,4647,1690.0,2014-09-01,0.363675,2014-01007,2950.456917,NaN,NaN,NaN
4,Alabama,Blount,01,009,01009,11385,4719.0,2014-09-01,0.414493,2014-01009,3086.813525,NaN,NaN,NaN


In [78]:
import plotly.plotly as py
from plotly.figure_factory._county_choropleth import create_choropleth

fips = ['06021', '06023', '06027',
        '06029', '06033', '06059',
        '06047', '06049', '06051',
        '06055', '06061']
values = range(len(fips))

fig = ff.create_choropleth(fips=fips, values=values)
py.iplot(fig, filename='choropleth of some cali counties - full usa scope')

ImportError: geopandas, pyshp and shapely must be installed for this figure factory.

Run the following commands to install the correct versions of the following modules:

```
pip install geopandas==0.3.0
pip install pyshp==1.2.10
pip install shapely==1.6.3
```
If you are using Windows, follow this post to properly install geopandas and dependencies:http://geoffboeing.com/2014/09/using-geopandas-windows/

If you are using Anaconda, do not use PIP to install the packages above. Instead use conda to install them:

```
conda install plotly
conda install geopandas
```